This codebook calculates performance metrics from the ChatGPT moral sentiment classifications using alternative prompts.

## Load Packages

In [12]:
import os
import pandas as pd
import pickle
import time

from sklearn.metrics import f1_score

## General Parameters

In [15]:
data = "mfrc"
mode = "full"
folder = "../data/preprocessed/"

alt = "1" #if multiple alternative prompts were created, change the value here (or iterate all code over a list of alternatives)
path = folder + data + "_sample_" + mode + ".csv"
pred_path =  "../results/predictions/gpt_" + data + "_labels_" + mode + "_ALT" + alt + ".csv"

# load ground truth and metadata
df_groundtruth = pd.read_csv("../data/preprocessed/" + data + "_sample_" + mode + ".csv")   
df_meta = pd.read_csv("../data/preprocessed/" + data + "_meta_sample_" + mode + ".csv")   
df_pred = pd.read_csv(pred_path)
df_pred.loc[(df_pred.loc[:, df_pred.columns[1:]].sum(1) == 0), "non-moral"] = 1 # non-moral if no moral sentiment

cols = df_pred.columns[1:].tolist()

## Calculate performance and save for bias analyses

In [16]:
y_true = df_groundtruth[cols].values
y_pred = df_pred[cols]

print(f1_score(y_true, y_pred, average="macro"))

df_total = df_meta.merge(df_pred[cols + ["text"]], on="text")
df_total["success"] = df_total.apply(lambda x: x[x["annotation"]] == 1, axis = 1)

df_total.to_csv("../results/evals/gpt_" + data + "_success_" + mode + "_" + alt + ".csv", index = False) # uncomment for analyses

0.2338934684578574


## Additional Information (predicted class distributions)

### Distribution of foundations across predictions

In [21]:
print(df_total[cols].sum(0)/df_total.shape[0])

care               0.085056
proportionality    0.011868
loyalty            0.026802
authority          0.148947
purity             0.007319
equality           0.057264
thin morality      0.657798
non-moral          0.004945
dtype: float64


#### Distribution of foundations for groundtruth

In [22]:
# compare distribution of foundations over predictions and groundtruth (are they similar? -> better classifier)
print(df_groundtruth[cols].sum(0)/df_groundtruth.shape[0])

care               0.113309
proportionality    0.037881
loyalty            0.035870
authority          0.048274
purity             0.018773
equality           0.077774
thin morality      0.095877
non-moral          0.646329
dtype: float64
